In [1]:
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
import os
os.environ['PYTHONWARNINGS'] = "ignore"
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import pickle
import xgboost as xgb
from shap import GPUTreeExplainer
from matplotlib.ticker import MaxNLocator,MultipleLocator
from scipy.stats import kendalltau
from scipy.special import expit

import mlresearch
mlresearch.utils.set_matplotlib_style()
from mlresearch.utils import set_matplotlib_style
set_matplotlib_style(font_size=27)

In [2]:
import numpy as np
print(np.__version__) #1.26.4
# print(shap.__version__) #0.46.1.dev86
print(sklearn.__version__) #1.6.0
print(xgb.__version__) #1.7.6

2.0.2
1.6.1
2.1.4


## Preprocessing

In [3]:
# Use White alone & African American only 
FEAT_CNT = 16
STATE = 'VA'
FOLDS = 5
seeds = [0,21,42,63,84]

In [4]:
# categorical_cols =['Occupation', 'Marriage','Place of Birth','Sex', 'Race']

with open(file=f'dataset/ACS_PublicCoverage_{STATE}.pickle', mode='rb') as f:
    df=pickle.load(f)
columns = df.columns
with pd.option_context('future.no_silent_downcasting', True):
    df['Sex'].replace( {'Female':0.0},inplace = True)
    df['Sex'].replace({'Male':1.0}, inplace = True)
    df['Nativity'].replace( {'Native':1.0},inplace = True)
    df['Nativity'].replace({'Foreign born':0.0}, inplace = True)
    df['Disability'].replace( {'No':0.0},inplace = True)
    df['Disability'].replace({'Yes':1.0}, inplace = True)
    df['Hearing Difficulty'].replace( {'No':0.0},inplace = True)
    df['Hearing Difficulty'].replace({'Yes':1.0}, inplace = True)
    df['Vision Difficulty'].replace( {'No':0.0},inplace = True)
    df['Vision Difficulty'].replace({'Yes':1.0}, inplace = True)
    df['Cognitive Difficulty'].replace( {'No':0.0},inplace = True)
    df['Cognitive Difficulty'].replace({'Yes':1.0}, inplace = True)
    
X = df.iloc[:, 0:FEAT_CNT]
Y = df.iloc[:, FEAT_CNT]

category_col =['Ancestry recode', 'Citizenship', 'Marriage','Employment Status','Mobility status', 'Military Service','Race']
X = pd.get_dummies(X, columns=category_col, drop_first=True)
for c in X.columns:
    X[c] = X[c].astype(float)

In [5]:
np.random.seed(0)
X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

## Utils

In [6]:
def get_bins(strategy, bin_size, seed):
    ## compute bin boundary
    np.random.seed(seed)
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
    for train_val_idx, test_idx in splitter.split(X, Y):
        X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
        Y_train_val, Y_test = Y.iloc[train_val_idx], Y.iloc[test_idx]
    
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
    for train_idx, val_idx in splitter.split(X_train_val, Y_train_val):
        X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
        Y_train, Y_val = Y_train_val.iloc[train_idx], Y_train_val.iloc[val_idx] 
    kd = KBinsDiscretizer(n_bins=bin_size, encode='ordinal', strategy=strategy)

    kd.fit(X_train)
    bin_boundaries = kd.bin_edges_[0]

    return bin_boundaries
def assign_age(age,bin_edges):
    # Assign age to a median of the bin_edges
    for idx in range(len(bin_edges)-1):
        if age == bin_edges[-1]:
            median = (bin_edges[-1] + bin_edges[-2])/2
        elif bin_edges[idx] <= age and age < bin_edges[idx+1]:
            median = (bin_edges[idx] + bin_edges[idx+1])/2
    return median

In [7]:
def compute_shap(X_train,X_test,Y_train,Y_test, seed):

    print('**********START**********')
    # Train model on new data
    param_grid = {
        'classifier__n_estimators': [50, 100, 200],  # Number of boosting rounds
        'classifier__max_depth': [3, 5, 7,9,11],          # Maximum tree depth
        'classifier__learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage 
        'classifier__colsample_bytree': [0.8, 1.0],  # Subsample ratio of columns for each tree
        'classifier__gamma': [0, 0.1, 0.2],          # Minimum loss reduction for a split
    }
    model = xgb.XGBClassifier(random_state=seed)
    grid_search = GridSearchCV(
        model, 
        param_grid,              # 3-fold cross-validation
        scoring='f1',   # Evaluation metric
        n_jobs=-1,            # Use all processors
        verbose=1             # Print progress
    )

    grid_search.fit(X_train, Y_train)
        
    # Extract the best model
    model = grid_search.best_estimator_

    explainer = GPUTreeExplainer(model,X_train,feature_perturbation = 'interventional')
    shap_values = explainer(X_test)
    pred = best_model.predict(X_test)
    return shap_values, pred
    
def get_tfs(shap_vals,Y_true, pred):
    # Compute indices of errors
    TP_i = np.where((Y_true == 1.0) & (pred == 1.0))[0]  # True Positives
    FP_i = np.where((Y_true == 0.0) & (pred == 1.0))[0]  # False Positives
    TN_i = np.where((Y_true == 0.0) & (pred == 0.0))[0]  # True Negatives
    FN_i = np.where((Y_true == 1.0) & (pred == 0.0))[0]  # False Negatives
    return TP_i,FP_i,TN_i,FN_i

def get_ranks(shap_vals):

    # avg_shap = np.mean(np.abs(shap_vals), axis=0)
    # Compute rankings row-wise
    sorted_indices = np.argsort(-np.abs(shap_vals), axis=1)  # Indices of absolute values in descending order
    rank = np.empty_like(sorted_indices)             # Create an empty array of the same shape
    rows, cols = shap_vals.shape                            # Get the shape of the array
    rank[np.arange(rows)[:, None], sorted_indices] = np.arange(1, cols + 1)  # Assign ranks row-wise

    target_rank = rank[:,0]

    return rank,target_rank

def compare_shap(s1, s2, plot = False):
    shap_dif = np.abs(np.subtract(s1[:, 0], s2[:, 0])) #/ np.abs(s1[:, 0])
    return shap_dif

def compare_ranks(rank1, rank2,agreed_idx, plot = False):

    age_ranks1,age_ranks2 = rank1[:,0],rank2[:,0]
    kendalls = [kendalltau(r1,r2,method='exact').statistic for r1,r2 in zip(rank1,rank2)]
    full_abs_rank_dif = [abs(r1-r2) for r1,r2 in zip(age_ranks1,age_ranks2)]
    full_rank_dif = [r1-r2 for r1,r2 in zip(age_ranks1,age_ranks2)]
    if plot:
        plot_frequency(full_rank_dif)
    top_4_idx = list()
    # Compute indices where rank differences are the largest and the smallest
    largest = list(np.argsort(full_rank_dif)[-100:][::-1])
    smallest = list(np.argsort(full_rank_dif)[:100])
    for s in smallest:
        if len(top_4_idx) < 4 and s in agreed_idx:
            top_4_idx.append(s)
    return full_rank_dif, full_abs_rank_dif, top_4_idx
    
def compare_results(s1, s2, rank1, rank2,agreed_idx, plot = False):
    # Compare baselines
    s1,s2 = s1.values, s2.values
    
    shap_dif = compare_shap(s1,s2)
    full_rank_dif, full_abs_rank_dif, top_4_idx = compare_ranks(rank1,rank2,agreed_idx,plot=plot)
    kendalls = [kendalltau(r1,r2,method='exact').statistic for r1,r2 in zip(rank1,rank2)]
        

    return top_4_idx, shap_dif, full_rank_dif, kendalls



def compute_fidelity(pred, sv, base):

    sv_sums = expit(np.sum(sv, axis=1)+base)
    binary_predictions = (sv_sums > 0.5).astype(float)
    fidelity = np.mean(binary_predictions == pred)
    match_idx = np.where(binary_predictions == pred)[0]
        
    return fidelity,match_idx

## Train the model with plain Age

In [8]:
# base_models = list()
base_shap_vals = list()
base_preds = list()
base_accs = list() 
base_f1s = list()
base_ranks = list()
base_age_ranks = list()

base_tp_idx = list()
base_fp_idx = list()
base_tn_idx = list()
base_fn_idx = list()

base_tp_age_ranks = list()
base_fp_age_ranks = list()
base_tn_age_ranks = list()
base_fn_age_ranks = list()

base_firsts = list()
base_percentages = list()
for seed in tqdm(seeds):
    np.random.seed(seed)
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
    for train_val_idx, test_idx in splitter.split(X, Y):
        X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
        Y_train_val, Y_test = Y.iloc[train_val_idx], Y.iloc[test_idx]
    
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
    for train_idx, val_idx in splitter.split(X_train_val, Y_train_val):
        X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
        Y_train, Y_val = Y_train_val.iloc[train_idx], Y_train_val.iloc[val_idx]
    param_grid = {
        'classifier__n_estimators': [50, 100, 200],  # Number of boosting rounds
        'classifier__max_depth': [3, 5, 7,9,11],          # Maximum tree depth
        'classifier__learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage 
        'classifier__colsample_bytree': [0.8, 1.0],  # Subsample ratio of columns for each tree
        'classifier__gamma': [0, 0.1, 0.2],          # Minimum loss reduction for a split
    }
    model = xgb.XGBClassifier(random_state=seed)
    grid_search = GridSearchCV(
        model, 
        param_grid,              # 3-fold cross-validation
        scoring='f1',   # Evaluation metric
        n_jobs=13,            # Use all processors
        verbose=1             # Print progress
    )

    grid_search.fit(X_train, Y_train)
    
    # Extract the best model
    best_model = grid_search.best_estimator_
    explainer = GPUTreeExplainer(best_model,X_train, feature_perturbation='interventional') 
    shap_values = explainer(X_test)
    
    sv = shap_values.values
    base_rank,age_rank= get_ranks(sv)
    base_ranks.append(base_rank)
    
    pred = best_model.predict(X_test)
    # base_models.append(best_model)
    base_shap_vals.append(shap_values)
    base_preds.append(pred)
    base_accs.append(accuracy_score(Y_test,pred)*100)
    base_f1s.append(f1_score(Y_test,pred)*100)
    base_ranks.append(base_rank)
    base_age_ranks.append(age_rank)

print(f'Overall average acc: {sum(base_accs)/len(base_accs):.2f} average f1s : {sum(base_f1s)/len(base_f1s):.2f}')


  0%|                                                       | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


 20%|█████████▍                                     | 1/5 [00:23<01:33, 23.30s/it]

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


 40%|██████████████████▊                            | 2/5 [00:38<00:56, 18.78s/it]

...

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


 80%|█████████████████████████████████████▌         | 4/5 [01:09<00:16, 16.38s/it]

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


100%|███████████████████████████████████████████████| 5/5 [01:24<00:00, 16.90s/it]

Overall average acc: 86.41 average f1s : 58.29


In [9]:
import pickle
from pathlib import Path
path = './results'
if not os.path.exists(path):
   # Create a new directory because it does not exist
   os.makedirs(path)

# # save
with open(path + '/Sens_PC_base_shap_vals_cv.pickle', 'wb') as f:
    pickle.dump(base_shap_vals, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Sens_PC_base_accs_cv.pickle', 'wb') as f:
    pickle.dump(base_accs, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Sens_PC_base_f1s_cv.pickle', 'wb') as f:
    pickle.dump(base_f1s, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Sens_PC_base_age_ranks_cv.pickle', 'wb') as f:
    pickle.dump(base_age_ranks, f, pickle.HIGHEST_PROTOCOL)
    
# with open(path + '/Sens_Income_base_tp_age_ranks_cv.pickle', 'wb') as f:
#     pickle.dump(base_tp_age_ranks, f, pickle.HIGHEST_PROTOCOL)
# with open(path + '/Sens_Income_base_fp_age_ranks_cv.pickle', 'wb') as f:
#     pickle.dump(base_fp_age_ranks, f, pickle.HIGHEST_PROTOCOL)
# with open(path + '/Sens_Income_base_tn_age_ranks_cv.pickle', 'wb') as f:
#     pickle.dump(base_tn_age_ranks, f, pickle.HIGHEST_PROTOCOL)
# with open(path + '/Sens_Income_base_fn_age_ranks_cv.pickle', 'wb') as f:
#     pickle.dump(base_fn_age_ranks, f, pickle.HIGHEST_PROTOCOL)

# with open(path + '/Sens_Income_base_firsts_cv.pickle', 'wb') as f:
#     pickle.dump(base_firsts, f, pickle.HIGHEST_PROTOCOL)
# with open(path + '/Sens_Income_base_percentages_cv.pickle', 'wb') as f:
#     pickle.dump(base_percentages, f, pickle.HIGHEST_PROTOCOL)

 # Equi Width

In [10]:
ew_fids = list()
ew_preds = list()
ew_ranks = list()
ew_age_ranks = list()
ew_shap_vals = list()
ew_rank_difs = list()

ew_tp_ranks = list()
ew_fp_ranks = list()
ew_tn_ranks = list()
ew_fn_ranks = list()

ew_firsts = list()
ew_percentages = list()
ew_first_rank_difs = list()

for bucket in range(2,21):
    b_fids = list()
    b_preds = list()
    b_ranks = list()
    b_age_ranks = list()
    b_shap_vals = list()
    b_rank_difs = list()

    b_tp_ranks = list()
    b_fp_ranks = list()
    b_tn_ranks = list()
    b_fn_ranks = list()

    b_firsts = list()
    b_percentages = list()
    b_first_rank_difs = list()
    for i, seed in enumerate(seeds):
        
        X2 = X.copy()
        bucket_edge = get_bins('uniform',bucket,seed)
        X2['Age'] = X2['Age'].apply(lambda age: assign_age(age, bucket_edge)) 
        np.random.seed(seed)
        splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
        for train_val_idx, test_idx in splitter.split(X2, Y):
            X_train_val, X_test = X2.iloc[train_val_idx], X2.iloc[test_idx]
            Y_train_val, Y_test = Y.iloc[train_val_idx], Y.iloc[test_idx]
        
        splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
        for train_idx, val_idx in splitter.split(X_train_val, Y_train_val):
            X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
            Y_train, Y_val = Y_train_val.iloc[train_idx], Y_train_val.iloc[val_idx] 

        
        # compute bin edges as shap values for bucketized data
        
        bucket_shap,bucket_pred = compute_shap(X_train,X_test,Y_train,Y_test,seed)
        sv = bucket_shap.values
        ew_rank, ew_age_rank = get_ranks(sv)
        # Compute fidelity and indices where explanation is the same.

        preds = base_preds[i]
        sv = bucket_shap.values
        base = bucket_shap.base_values
        ew_fid, ew_agreed = compute_fidelity(preds,sv,base)

        b_fids.append(ew_fid)
        b_preds.append(bucket_pred)
        b_ranks.append(ew_rank)
        b_age_ranks.append(ew_age_rank)
        b_shap_vals.append(bucket_shap)

        
        # Compuute rank shift
        base_age_rank = base_age_ranks[i]
        rank_dif = [r1-r2 for r1,r2 in zip(base_age_rank,ew_age_rank)]
        b_rank_difs.append(rank_dif)

        
    ew_fids.append(b_fids)
    ew_preds.append(b_preds)
    ew_ranks.append(b_ranks)
    ew_age_ranks.append(b_age_ranks)
    ew_shap_vals.append(b_shap_vals)
    ew_rank_difs.append(b_rank_difs)


**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********

In [11]:
# # save
with open(path + '/Sens_PC_ew_fids_cv.pickle', 'wb') as f:
    pickle.dump(ew_fids, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Sens_PC_ew_ranks_cv.pickle', 'wb') as f:
    pickle.dump(ew_ranks, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Sens_PC_ew_age_ranks_cv.pickle', 'wb') as f:
    pickle.dump(ew_age_ranks, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Sens_PC_ew_shap_vals_cv.pickle', 'wb') as f:
    pickle.dump(ew_shap_vals, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Sens_PC_ew_rank_difs_cv.pickle', 'wb') as f:
    pickle.dump(ew_rank_difs, f, pickle.HIGHEST_PROTOCOL)

# with open(path + '/Sens_Income_ew_tp_ranks_ranks_cv.pickle', 'wb') as f:
#     pickle.dump(ew_tp_ranks, f, pickle.HIGHEST_PROTOCOL)
# with open(path + '/Sens_Income_ew_fp_age_ranks_cv.pickle', 'wb') as f:
#     pickle.dump(ew_fp_ranks, f, pickle.HIGHEST_PROTOCOL)
# with open(path + '/Sens_Income_ew_tn_age_ranks_cv.pickle', 'wb') as f:
#     pickle.dump(ew_tn_ranks, f, pickle.HIGHEST_PROTOCOL)
# with open(path + '/Sens_Income_ew_fn_age_ranks_cv.pickle', 'wb') as f:
#     pickle.dump(ew_fn_ranks, f, pickle.HIGHEST_PROTOCOL)

# with open(path + '/Sens_Income_ew_firsts_cv.pickle', 'wb') as f:
#     pickle.dump(ew_firsts, f, pickle.HIGHEST_PROTOCOL)
# with open(path + '/Sens_Income_ew_percentages_cv.pickle', 'wb') as f:
#     pickle.dump(ew_percentages, f, pickle.HIGHEST_PROTOCOL)
# with open(path + '/Sens_Income_ew_first_rank_difs_cv.pickle', 'wb') as f:
#     pickle.dump(ew_first_rank_difs, f, pickle.HIGHEST_PROTOCOL)

# Examples

In [ ]:
# # examples
# first_dict = dict()
# for idx,shift in enumerate(examples[0][0][0]):
#     if shift not in first_dict:
#         first_dict[shift] = [examples[0][1][0][idx]]
#     else:
#         first_dict[shift].append(examples[0][1][0][idx])
# smallest = min(list(first_dict.keys()))
# print(smallest)
# for it,i in enumerate(first_dict[smallest]):
#     shap.plots.bar(base_result['shap_vals'][0][i],max_display=22)
# for it,i in enumerate(first_dict[smallest]):
#     shap.plots.bar(example_results[0]['shap_vals'][0][i],max_display=22)   

In [ ]:
# # TPS
# first_dict = dict()
# for idx,shift in enumerate(examples[1][0][0]):
#     if shift not in first_dict:
#         first_dict[shift] = [examples[1][1][0][idx]]
#     else:
#         first_dict[shift].append(examples[1][1][0][idx])
# smallest = min(list(first_dict.keys()))
# print(smallest)
# for it,i in enumerate(first_dict[smallest]):
#     shap.plots.bar(base_result['shap_vals'][0][tps[0][i]],max_display=22)
# for it,i in enumerate(first_dict[smallest]):
#     shap.plots.bar(example_results[0]['shap_vals'][0][tps[0][i]],max_display=22)  

In [ ]:
# # fPS
# first_dict = dict()
# for idx,shift in enumerate(examples[2][0][0]):
#     if shift not in first_dict:
#         first_dict[shift] = [examples[2][1][0][idx]]
#     else:
#         first_dict[shift].append(examples[2][1][0][idx])
# smallest = min(list(first_dict.keys()))
# print(smallest)
# for it,i in enumerate(first_dict[smallest]):
#     shap.plots.bar(base_result['shap_vals'][0][fps[0][i]],max_display=22)
# for it,i in enumerate(first_dict[smallest]):
#     shap.plots.bar(example_results[0]['shap_vals'][0][fps[0][i]],max_display=22)  

In [ ]:
# # tnS
# first_dict = dict()
# for idx,shift in enumerate(examples[3][0][0]):
#     if shift not in first_dict:
#         first_dict[shift] = [examples[3][1][0][idx]]
#     else:
#         first_dict[shift].append(examples[3][1][0][idx])
# smallest = min(list(first_dict.keys()))
# print(smallest)
# for it,i in enumerate(first_dict[smallest]):
#     shap.plots.bar(base_result['shap_vals'][0][tns[0][i]],max_display=22)
# for it,i in enumerate(first_dict[smallest]):
#     shap.plots.bar(example_results[0]['shap_vals'][0][tns[0][i]],max_display=22)  

In [ ]:
# # fnS
# first_dict = dict()
# for idx,shift in enumerate(examples[4][0][0]):
#     if shift not in first_dict:
#         first_dict[shift] = [examples[4][1][0][idx]]
#     else:
#         first_dict[shift].append(examples[4][1][0][idx])
# smallest = min(list(first_dict.keys()))
# print(smallest)
# for it,i in enumerate(first_dict[smallest]):
#     shap.plots.bar(base_result['shap_vals'][0][fns[0][i]],max_display=22)
# for it,i in enumerate(first_dict[smallest]):
#     shap.plots.bar(example_results[0]['shap_vals'][0][fns[0][i]],max_display=22)  

In [ ]:
means = [m for m,s in ew_bucket_downs]
stds = [s for m,s in ew_bucket_downs]

decrease_bin = list()
for decrease_idx in range(4):
    m = [mean[decrease_idx] for mean in means] 
    decrease_bin.append(m)

colors = ['r','b','g','c','m']
p_labels = ['Rank 2','Rank 3','Rank 4', 'Rank 5 or below']

for i in range(len(decrease_bin)):
    plt.plot(list(range(2,21)), decrease_bin[i],color=colors[i],label=p_labels[i],ls='-')

plt.xticks(range(2,21))
plt.title('Rank shifts where Rank of Age is 1')
plt.xlabel('Number of Buckets')
plt.xlabel('Proportion')
plt.legend()

In [ ]:
means = [m for m,s in ew_bucket_tp_downs]
stds = [s for m,s in ew_bucket_tp_downs]

decrease_bin = list()
for decrease_idx in range(4):
    m = [mean[decrease_idx] for mean in means] 
    decrease_bin.append(m)

colors = ['r','b','g','c','m']
p_labels = ['Rank 2','Rank 3','Rank 4', 'Rank 5 or below']

for i in range(len(decrease_bin)):
    plt.plot(list(range(2,21)), decrease_bin[i],color=colors[i],label=p_labels[i],ls='-')

plt.xticks(range(2,21))
plt.title('TP Rank shifts where Rank of Age is 1')
plt.xlabel('Number of Buckets')
plt.xlabel('Proportion')
plt.legend()

In [ ]:
means = [m for m,s in ew_bucket_fp_downs]
stds = [s for m,s in ew_bucket_fp_downs]

decrease_bin = list()
for decrease_idx in range(4):
    m = [mean[decrease_idx] for mean in means] 
    decrease_bin.append(m)

colors = ['r','b','g','c','m']
p_labels = ['Rank 2','Rank 3','Rank 4', 'Rank 5 or below']

for i in range(len(decrease_bin)):
    plt.plot(list(range(2,21)), decrease_bin[i],color=colors[i],label=p_labels[i],ls='-')

plt.xticks(range(2,21))
plt.title('FP Rank shifts where Rank of Age is 1')
plt.xlabel('Number of Buckets')
plt.xlabel('Proportion')
plt.legend()

In [ ]:
means = [m for m,s in ew_bucket_tn_downs]
stds = [s for m,s in ew_bucket_tn_downs]

decrease_bin = list()
for decrease_idx in range(4):
    m = [mean[decrease_idx] for mean in means] 
    decrease_bin.append(m)

colors = ['r','b','g','c','m']
p_labels = ['Rank 2','Rank 3','Rank 4', 'Rank 5 or below']

for i in range(len(decrease_bin)):
    plt.plot(list(range(2,21)), decrease_bin[i],color=colors[i],label=p_labels[i],ls='-')

plt.xticks(range(2,21))
plt.title('TN Rank shifts where Rank of Age is 1')
plt.xlabel('Number of Buckets')
plt.xlabel('Proportion')
plt.legend()

In [ ]:
means = [m for m,s in ew_bucket_fn_downs]
stds = [s for m,s in ew_bucket_fn_downs]

decrease_bin = list()
for decrease_idx in range(4):
    m = [mean[decrease_idx] for mean in means] 
    decrease_bin.append(m)

colors = ['r','b','g','c','m']
p_labels = ['Rank 2','Rank 3','Rank 4', 'Rank 5 or below']

for i in range(len(decrease_bin)):
    plt.plot(list(range(2,21)), decrease_bin[i],color=colors[i],label=p_labels[i],ls='-')

plt.xticks(range(2,21))
plt.title('FN Rank shifts where Rank of Age is 1')
plt.xlabel('Number of Buckets')
plt.xlabel('Proportion')
plt.legend()